In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CDC_Employee_Project") \
    .config(
        "spark.jars",
        "/Users/manidharrao16/docs/manidocs/pyspark/jdbc/mysql-connector-j-8.4.0/mysql-connector-j-8.4.0.jar"
    ) \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/26 16:02:08 WARN Utils: Your hostname, Vardhinenis-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.31 instead (on interface en0)
26/01/26 16:02:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/26 16:02:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
jdbc_url = "jdbc:mysql://127.0.0.1:3306/company_db"
properties = {
    "user": "root",
    "password": "Manidharrao@777",
    "driver": "com.mysql.cj.jdbc.Driver"
}
employee_df = spark.read.jdbc(
    url=jdbc_url,
    table="employee_data",
    properties=properties
)
employee_df.show()

+------+--------+-------+---------+------+
|emp_id|emp_name|   dept|     city|salary|
+------+--------+-------+---------+------+
|   101|    Raju|     IT|Hyderabad| 50000|
|   102|  Anitha|     HR|Bangalore| 45000|
|   103|  Suresh|     IT|  Chennai| 55000|
|   104|   Priya|Finance|Hyderabad| 60000|
|   105|  Ramesh|     HR|Hyderabad| 42000|
|   106|   Kiran|     IT|Hyderabad| 52000|
|   107|    Neha|Finance|Hyderabad| 58000|
|   108|   Arjun|     IT|Bangalore| 54000|
|   109|   Divya|     HR|  Chennai| 46000|
|   110|    Amit|     IT|Hyderabad| 51000|
|   111|   Sneha|Finance|Hyderabad| 62000|
|   112|   Vikas|     IT|Hyderabad| 53000|
|   113|   Pooja|     HR|Hyderabad| 44000|
|   114|   Manoj|     IT|Bangalore| 56000|
|   115|   Kavya|Finance|Hyderabad| 59000|
|   116|  Sanjay|     HR|Hyderabad| 43000|
|   117|    Ritu|     IT|Hyderabad| 57000|
|   118|  Nikhil|     IT|Bangalore| 54500|
|   119|   Meena|     HR|Hyderabad| 45500|
|   120|   Varun|Finance|Bangalore| 61000|
+------+---

In [4]:
cdc_df = spark.read \
    .option("header", "true") \
    .csv("/Users/manidharrao16/docs/manidocs/pyspark/employees_cdc.csv")
cdc_df.show()

+------+--------+------+---------+---------+
|emp_id|emp_name|salary|     city|operation|
+------+--------+------+---------+---------+
|   102|  Anitha| 48000|Bangalore|   UPDATE|
|   107|    Neha| 60000|    Noida|   UPDATE|
|   119|   Meena| 47000| Warangal|   UPDATE|
|   126|  Rajesh| 42000|Hyderabad|   UPDATE|
|   127|   Suman| 50000|Bangalore|   UPDATE|
|   128| Karthik| 53000|  Chennai|   UPDATE|
|   105|  Ramesh|  NULL|     Pune|   DELETE|
|   113|   Pooja|  NULL|Hyderabad|   DELETE|
|   121|  Deepak|  NULL|  Chennai|   DELETE|
+------+--------+------+---------+---------+



In [5]:
# Separate CDC Operations (UPDATE / INSERT / DELETE)

from pyspark.sql.functions import col, coalesce, lit

updates_df = cdc_df.filter(col("operation") == "UPDATE")
inserts_df = cdc_df.filter(col("operation") == "INSERT")
deletes_df = cdc_df.filter(col("operation") == "DELETE")

In [6]:
# Apply UPDATE Logic
# Update salary and city only if present in CDC file.

updated_employee_df = employee_df.alias("emp") \
    .join(updates_df.alias("cdc"), "emp_id", "left") \
    .select(
        col("emp_id"),
        col("emp.emp_name"),
        col("emp.dept"),
        coalesce(col("cdc.city"), col("emp.city")).alias("city"),#take new value if available, else keep old value
        coalesce(col("cdc.salary"), col("emp.salary")).alias("salary")
    )
# coalesce() means: "take the first value that is NOT empty (NOT NULL)"

# If CDC file gives new city → update it
# If CDC file city is empty → keep old city from employee table
# coalesce(col("cdc.city"), col("emp.city")).alias("city")
# If CDC file gives new salary → update it
# If CDC file salary is empty → keep old salary from employee table
# coalesce(col("cdc.salary"), col("emp.salary")).alias("salary")

In [7]:
# Apply DELETE Logic
# Remove employees whose emp_id exists in DELETE records.
after_delete_df = updated_employee_df.join(
    deletes_df.select("emp_id"),
    on="emp_id",
    how="left_anti"
)
# left_anti = Give me rows from left side that do NOT exist in delete list

In [8]:
# Apply INSERT Logic
# Create new employee records.
new_employees_df = inserts_df.select(
    col("emp_id"),
    lit("New Employee").alias("emp_name"),
    lit("NA").alias("dept"),
    col("city"),
    col("salary")
)

In [9]:
# Final CDC Result
# Combine updated + inserted records.
final_df = after_delete_df.unionByName(new_employees_df)
final_df.show(30)

+------+--------+-------+---------+------+
|emp_id|emp_name|   dept|     city|salary|
+------+--------+-------+---------+------+
|   101|    Raju|     IT|Hyderabad| 50000|
|   102|  Anitha|     HR|Bangalore| 48000|
|   103|  Suresh|     IT|  Chennai| 55000|
|   104|   Priya|Finance|Hyderabad| 60000|
|   106|   Kiran|     IT|Hyderabad| 52000|
|   107|    Neha|Finance|    Noida| 60000|
|   108|   Arjun|     IT|Bangalore| 54000|
|   109|   Divya|     HR|  Chennai| 46000|
|   110|    Amit|     IT|Hyderabad| 51000|
|   111|   Sneha|Finance|Hyderabad| 62000|
|   112|   Vikas|     IT|Hyderabad| 53000|
|   114|   Manoj|     IT|Bangalore| 56000|
|   115|   Kavya|Finance|Hyderabad| 59000|
|   116|  Sanjay|     HR|Hyderabad| 43000|
|   117|    Ritu|     IT|Hyderabad| 57000|
|   118|  Nikhil|     IT|Bangalore| 54500|
|   119|   Meena|     HR| Warangal| 47000|
|   120|   Varun|Finance|Bangalore| 61000|
|   122|  Swathi|     HR|   Mumbai| 47000|
|   123|    Ajay|     IT|Hyderabad| 53500|
|   124| La

In [10]:
final_df.write.jdbc(
    url=jdbc_url,
    table="employee_target",
    mode="overwrite",
    properties=properties
)